#  Implementación en Python para el dataset "fashion-mnist"

Se importan las librerías necesarias y se crea un diccionario que relaciona las etiquetas numéricas con sus respectivos nombres.

In [ ]:
import datetime
from enum import Enum
import time
from sklearn.linear_model import LogisticRegression, SGDClassifier
import pickle
import os

CACHE_DIR = "cache"

TARGETS: dict[int, str] = {
    0: "t-shirt/top",
    1: "trouser",
    2: "pullover",
    3: "dress",
    4: "coat",
    5: "sandal",
    6: "shirt",
    7: "sneaker",
    8: "bag",
    9: "ankle boot",
}

Se crea un enum para poder reconocer de manera sencilla en los condicionales qué clasificador se desea utilizar.

In [ ]:
class Models(Enum):
    SGDC = 1
    LR = 2

Función obtenida de la documentación de fashion-mnist para poder cargar la información de prueba y entrenamiento del repositorio de github. Para que funcione, es necesario tener clonado el repositorio de github en la dirección de esta libreta de jupyter.

In [ ]:
def load_mnist(path, kind="train"):
    import os
    import gzip
    import numpy as np

    labels_path = os.path.join(path, "%s-labels-idx1-ubyte.gz" % kind)
    images_path = os.path.join(path, "%s-images-idx3-ubyte.gz" % kind)

    with gzip.open(labels_path, "rb") as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8, offset=8)

    with gzip.open(images_path, "rb") as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8, offset=16).reshape(
            len(labels), 784
        )

    return images, labels

Esta función se encarga, como su nombre lo indica, de crear u obtener el modelo deseado. La primerea vez que se corre la función, se lleva acabo el entrenamiento normal, mientras que en las siguientes llamadas a la función, se lee el modelo de los archivos. Esto es para poder correr la función múltiples veces sin tener que esperar el entrenamiento del modelo una y otra vez. En cas de que force_train sea verdadero, los archivos generados, en caso de existir, son ignorados, llevan acabo siempre el entrenamiento nuevamente.

De igual manera, los valores de entrada son normalizados a un valor entre 0 y 1, para que el entrenamiento sea más rápido, ya que los valores más pequeños logran que el modelo converja rápidamente.

In [ ]:
def create_or_load_model(
    selected_model: Models, force_train: bool = False
) -> LogisticRegression | SGDClassifier:
    file_suffix: str | None = None
    if selected_model == Models.SGDC:
        file_suffix = "SGDC"
    elif selected_model == Models.LR:
        file_suffix = "LR"

    if file_suffix is None:
        raise Exception

    filePath = os.path.join(CACHE_DIR, f"fashion_mnist_{file_suffix}")
    if not force_train and os.path.isfile(filePath):
        return pickle.load(open(filePath, "rb"))

    x_train, y_train = load_mnist("fashion-mnist/data/fashion/", kind="train")

    x_train = x_train / 255

    model: SGDClassifier | LogisticRegression | None = None
    if selected_model == Models.SGDC:
        model = SGDClassifier()
    elif selected_model == Models.LR:
        model = LogisticRegression(multi_class="ovr", max_iter=10**10)

    if model is None:
        raise Exception

    model.fit(x_train, y_train)

    pickle.dump(model, open(filePath, "wb"))

    return model

Función de librearía para obtener los datos de prueba. Los datos entreada son normalizados a valores entre 0 y 1, para que el modelo converja más rápido.

In [ ]:
x_test, y_test = load_mnist("fashion-mnist/data/fashion", kind="t10k")
x_test = x_test / 255

Se llama la función para cargar o crear el modelo. Igual se lleva el tiempo que toma la función en correr, para poder comparar el tiempo de entrenamiento entre clasificadores.

In [ ]:
start = time.perf_counter()
model = create_or_load_model(Models.LR, force_train=True)
end = time.perf_counter()

Se predicen los valores con el modelo entrenado y se calcula el porcentaje de predicciones correctas.

In [ ]:
y_predicted = model.predict(x_test)
                                                                                              
success_log: list[bool] = []
for predicted, expected in zip(y_predicted, y_test):
    if predicted == expected:
        success_log.append(True)
    else:
        success_log.append(False)
                                                                                              
print(
    f"LogisticRegression model's training took {datetime.timedelta(seconds=(end - start))}"
)
print(
    f"LogisticRegression model's success rate: {success_log.count(True)  / len(success_log)}"
)

Se llama la función para cargar o crear el modelo. Igual se lleva el tiempo que toma la función en correr, para poder comparar el tiempo de entrenamiento entre clasificadores.

In [ ]:
start = time.perf_counter()
model = create_or_load_model(Models.SGDC, force_train=True)
end = time.perf_counter()

Se predicen los valores con el modelo entrenado y se calcula el porcentaje de predicciones correctas.

In [1]:
y_predicted = model.predict(x_test)
                                                                                              
success_log: list[bool] = []
for predicted, expected in zip(y_predicted, y_test):
    if predicted == expected:
        success_log.append(True)
    else:
        success_log.append(False)
                                                                                              
print(
    f"SGDClassifier model's training took {datetime.timedelta(seconds=(end - start))}"
)
print(
    f"SGDClassifier model's success rate: {success_log.count(True)  / len(success_log)}"
)

LogisticRegression model's training took 0:03:59.121519
LogisticRegression model's success rate: 0.841
SGDClassifier model's training took 0:00:21.746952
SGDClassifier model's success rate: 0.8315


Como se puede obtener en la impresión anterior, el porcentaje de precciones correctas de ambos modelos es bastante cercano, mientras que el tiempo de entrenamiento de LogisticRegression es mucho mayor al del SGDClassifier. Esto quiere decir que, para este caso de uso, el SGDClassifier es el mejor modelo de ambos.